# Cloud sandbox powered by FIWARE

## About the open source GPL3 license and copyright for this product

Copyright (c) 2024 Computate Limited Liability Company in Utah, USA

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

ADDITIONAL TERMS

As stated in section 7. c) and e) of the GPL3 license, 
"you may supplement the terms of this License with terms," 
Computate has added the following additional terms to the license: 

  7 c) Prohibiting misrepresentation of the origin of that material, and
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version;

  7 e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks;

Please do not redistribute this course until you have built your own platform with these tools, 
separate from the computate.org platform, and reconfigure your fork of this repo to deploy 
your own platform instead of the computate.org platform. 

QUESTIONS

For questions about this open source license, please contact our public mailing list at computate@group.computate.org


## Switch to the Python 3.9 Kernel

When opening this Jupyter Notebook in VSCode, you will need to switch to the Jupyter 3.9 Kernel. 

## About the Apache Solr search engine

> I asked my search engine for a good joke. It said, "A man walked into a bar, tavern, pole, inn, stake, tap room, stick, pub, shaft, parlour, shank, watering hole, rail, cantina, beam, alehouse, spoke, or saloon... but most likely a bar, and said ouch." 

An open source search engine like Apache Solr is required by the Smart Village application API to serve up API stored objects as quickly as possible. APIs that are backed by a search engine have numerous additional benefits compared to an API backed by a traditional relational database. A search engine is always indexed for the fastest data retrieval possible. The advanced data and query parsing of a search engine allows for extremely fast full text search, filtering, and sorting of the data. Search engine data can also be grouped, faceted, and pivoted on, for an advanced set of analytics and statistics on specific data in your query. 


## Deploy Solr with Helm


Run the command below to deploy the default computate Solr ConfigSet as a Kubernetes ConfigMap to the cloud. These configsets will be loaded later, when the Solr pod is running to initialize the Solr ConfigSet which is the schema you can use for Solr Collections in your site. 

Run the commands below to install solr with helm. 

In [ ]:
%%bash
helm upgrade --install solr oci://registry-1.docker.io/bitnamicharts/solr \
  --set image.tag=9.6.1 \
  --set-json 'resources={"requests": {"cpu": "200m", "memory": "500Mi"}}' \
  --set ingress.enabled=false \
  --set zookeeper.enabled=false \
  --set externalZookeeper.servers={zookeeper:2181} \
  --set persistence.size=2Gi \
  --set collectionShards=1 \
  --set collectionReplicas=1 \
  --set replicaCount=1
echo DONE

### View solr pod details
After deploying solr, it will take a minute before the solr pod is up and running. 
Run the command below until the solr pod health checks are `READY 1/1` and `STATUS Running`. 

### View solr pod details
After deploying solr, it will take a minute before the solr pod is up and running. 
Run the command below until the solr pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc get pod -l app.kubernetes.io/name=solr
oc wait pod -l app.kubernetes.io/name=solr --for=condition=Ready --timeout=5m
oc get pod -l app.kubernetes.io/name=solr
echo DONE

## Upload default computate Solr configset
A Solr configset is a data schema for a search engine. It specifies the exact fields and their data types that are available in the search engine to use.

In [ ]:
%%bash
oc exec pod/solr-0 -- mkdir -p /bitnami/solr/backup
oc rsync solr/ pod/solr-0:/bitnami/solr/backup/
oc exec pod/solr-0 -- curl -k -s -X POST -u "admin:$(oc get secret/solr -o jsonpath={.data.solr-password} | base64 -d)" \
  -H "Content-Type:application/octet-stream" \
  --data-binary @/bitnami/solr/backup/computate-configset.zip \
  "http://localhost:8983/solr/admin/configs?action=UPLOAD&name=computate&overwrite=true"
echo DONE

## Create a computateorg search collection
Create a Solr search collection named `computateorg` that we will use for 
the API of the Smart Device website powered by FIWARE we will use later. 

In [ ]:
%%bash
oc exec pod/solr-0 -- \
  bash -c 'env SOLR_AUTH_TYPE=basic SOLR_AUTHENTICATION_OPTS="-Dbasicauth=$SOLR_ADMIN_USERNAME:$SOLR_ADMIN_PASSWORD" /opt/bitnami/solr/bin/solr create_collection -c computateorg -n computate'
echo DONE

## View Solr admin password
The username will be `admin` and run the command below to find the admin password.

In [ ]:
%%bash
oc get secret/solr -o jsonpath={.data.solr-password} | base64 -d

## Next...
If you have successfully ran all of the commands above, congratulations, you are ready to move on to the next notebook in the course. 

- If you have additional questions or issues, please [create an issue for the course here](https://github.com/computate-org/computate/issues). 
- Otherwise, please continue to the next document [04-deploy-postgres.ipynb](04-deploy-postgres.ipynb). 
